# Homework 5 - Visit the Wikipedia hyperlinks graph!

# Libraries

In [1]:
import pandas as pd
from collections import defaultdict
import networkx as nx
import pickle
import collections
import math
from tqdm import tqdm
import numpy as np
from collections import defaultdict
import statistics
import operator
import random

### Researh question 1

[RQ1] Build the graph [G=(V, E)] , where V is the set of articles and E the hyperlinks among them, and provide its basic information:

* If it is direct or not
* The number of nodes
* The number of edges
* The average node degree. Is the graph dense?

# Data

In [2]:
# Reading reduced edges file as dataframe, where one column is the source and other is the destination
source_destination=pd.read_csv('wiki-topcats-reduced.txt',sep="\t",header=None,names=["source","destination"])

>reading of a dicionary created in _Homework 5 - RQ1 pre-check up.ipynb_ notebook file
source_destination_dict is a **default dictionary** with list as datatype for value of dictionary, where **key**=source_id(article_id), **value**=destination_id(list of article_id's which are connected to that article)

#reading pickle file into a memory, which contains a dictionary where **key=source_id(article_id)**, **value=destination_id(list of article_id's** which are connected to that article)

In [3]:
source_destination_dict = pickle.load(open('source_destination_dict.p', 'rb'))

In [4]:
#list of (source, destination) tuples which will be used to read edges from them in networkx _add_edges_from_ method
source_destination_tuples=[(row["source"],row["destination"]) for idx,row in tqdm(source_destination.iterrows())]

2645247it [03:07, 14115.72it/s]


### The number of nodes and edges

Based on the results and the conclusion from the _Homework 5 - RQ1 pre-check up.ipynb_ we can use DiGraph() method to make a directed graph immediately. And check again the subquestions of the RQ1 in order to give the final response and conclusions.

In [5]:
Gtup_directed = nx.DiGraph()
Gtup_directed.add_edges_from(source_destination_tuples)
print("Number of nodes:",len(Gtup_directed.nodes()),"\nNumber of edges:",len(Gtup_directed.edges()))

Number of nodes: 461193 
Number of edges: 2645247


### Average node degree

In [6]:
print(nx.info(Gtup_directed))

Name: 
Type: DiGraph
Number of nodes: 461193
Number of edges: 2645247
Average in degree:   5.7357
Average out degree:   5.7357


###  Is the graph dense?

In [7]:
print('Density of the graph:',nx.density(Gtup_directed))

Density of the graph: 1.2436602635647606e-05


In [8]:
#The graph is a sparse graph. Because the density has a small value.

Results of the RQ1:
    
>The graph is **DIRECTED**. That is why we used DiGraph() method from nx library
 
>The **number of nodes**: 461 193

>The **number of edges**: 2 645 247

>The **average node degree** is: 5.7357

>Is the graph dense? Density = 1.2437e-05 The density can have value from 0 to 1, whereby it is 0 for a graph without edges and 1 for a complete graph. Therefore we can conclude that **the graph is sparse** and not dense.

### Researh question 2

1. Building Block Ranking

    > Based on the implementation of the **shortest path** algorithm compare sample number of nodes of C0-input category with all nodes in all the other Ci categories in order to build the **block ranking**. 

2. Ranking nodes of each category in the block ranking vector

In [9]:
#categories file, where each category has a list of articles connected to it
categories=pd.read_csv('wiki-topcats-categories.txt',sep="\n",header=None)

In [10]:
def num_cat(row):
   
    """
    method which performs cleaning of each row from the categories file and counting the actual number of articles that belong to that category
  
    returns the number of articles 
    """
    return len(row.split("; ")[1].split(" "))

In [11]:
#applying the lambda and num_cat method for labeling and selecting categories who have more than 3500 articles
categories=categories[categories.iloc[:,0].apply(lambda x:True if num_cat(x)>3500 else False)]

In [12]:
#cleaning the dataframe with categories and mapping the articles to integers.
#and making the categories_dict where key=category name, value=list of article id's belonging to that category
categories_dict=defaultdict(list)
for idx in categories.index:
    cat_and_values=categories[0].loc[idx].split("; ")
    cat_name=cat_and_values[0].split(":")[1]
    categories_dict[cat_name]=list(map(int,cat_and_values[1].split(" ")))

In [13]:
# We should consider as nodes, only the ones given in the reduced version of the graph. Therefore we are checking and removing
#the other ones and based on that 6 more categories are eliminated. Therefore, now we have 29 categories.
categories = {}
all_nodes=set(Gtup_directed.nodes())
for key, values in categories_dict.items():
        categories[key] = all_nodes.intersection(set(values))
        if len(categories[key]) < 3500:
            del(categories[key])

>In order to test and make a runnable version of our code, we decided to test it on the sample. We have chosen the **'Windows_games'** as our **input category** and we will perform random sample, which is ok because it is a really sparse graph, and we decided select **100 nodes** from the input category and **all the other nodes** from the rest of the categories.

In [144]:
cat_edges={}
for cat in categories:
    cat_edges[cat]=len(Gtup_directed.subgraph(categories[cat]).edges)

In [145]:
cat_edges

{'English_footballers': 18233,
 'The_Football_League_players': 19672,
 'Association_football_forwards': 4710,
 'Association_football_goalkeepers': 8209,
 'Association_football_midfielders': 3801,
 'Association_football_defenders': 2286,
 'Living_people': 1218406,
 'Harvard_University_alumni': 3959,
 'Major_League_Baseball_pitchers': 10472,
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 30039,
 'Indian_films': 3323,
 'Year_of_death_missing': 1242,
 'Year_of_birth_missing_(living_people)': 6814,
 'Rivers_of_Romania': 15241,
 'Main_Belt_asteroids': 10891,
 'Asteroids_named_for_people': 142,
 'English-language_albums': 7064,
 'British_films': 2805,
 'English-language_films': 21278,
 'American_films': 10372,
 'People_from_New_York_City': 3156,
 'American_television_actors': 32231,
 'American_film_actors': 53164,
 'Debut_albums': 779,
 'Black-and-white_films': 6288,
 'Year_of_birth_missing': 1112,
 'Place_of_birth_missing_(living_people)': 736,
 'American_military_pe

In [149]:
sorted(cat_edges.items(), key=operator.itemgetter(0), reverse= False)

[('American_film_actors', 53164),
 ('American_films', 10372),
 ('American_military_personnel_of_World_War_II', 5133),
 ('American_television_actors', 32231),
 ('Association_football_defenders', 2286),
 ('Association_football_forwards', 4710),
 ('Association_football_goalkeepers', 8209),
 ('Association_football_midfielders', 3801),
 ('Asteroids_named_for_people', 142),
 ('Black-and-white_films', 6288),
 ('British_films', 2805),
 ('Debut_albums', 779),
 ('English-language_albums', 7064),
 ('English-language_films', 21278),
 ('English_footballers', 18233),
 ('Harvard_University_alumni', 3959),
 ('Indian_films', 3323),
 ('Living_people', 1218406),
 ('Main_Belt_asteroids', 10891),
 ('Major_League_Baseball_pitchers', 10472),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
  30039),
 ('People_from_New_York_City', 3156),
 ('Place_of_birth_missing_(living_people)', 736),
 ('Rivers_of_Romania', 15241),
 ('The_Football_League_players', 19672),
 ('Windows_games', 1),
 ('Ye

In [159]:
#random sampling of 100 nodes from the input category 'Windows_games' and making a sample subgraph for it
for i in range(100):
    cat_nodes_lst = random.sample(categories['American_film_actors'], 100)
    Gtup_directed_sample_C0=Gtup_directed.subgraph(cat_nodes_lst)
    if len(Gtup_directed_sample_C0.subgraph(categories['American_film_actors']).edges)>10:
        break

In [160]:
len(Gtup_directed.subgraph(categories['American_film_actors']).edges)

53164

In [161]:
nx.info(Gtup_directed_sample_C0)

'Name: \nType: SubDiGraph\nNumber of nodes: 100\nNumber of edges: 13\nAverage in degree:   0.1300\nAverage out degree:   0.1300'

In [162]:
def bfs_shortest_path(graph, start):
    """
    method which returns shorthest paths dictionary where key=node, and value is the actual distance of the shortest path from the start node to the other nodes of the graph
    """
    explored = set()
    queue = [start]
    shortest_path_dict = {}
    counter_level = 0
    to_visit_list = []
    while queue:
        for node in queue:
            if node not in explored:
                shortest_path_dict[node] = counter_level
                neighbours = graph[node]
                explored.add(node)
                to_visit_list.extend(neighbours)
        queue = []
        queue.extend(to_visit_list)
        to_visit_list = []
        counter_level += 1
    return shortest_path_dict

In [163]:
#calculate shorthest path from each article of the Graph input category sample to all the other nodes in the Graph
l = []
for article in tqdm(Gtup_directed_sample_C0.nodes()):
    l.append(bfs_shortest_path(Gtup_directed, article))   

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:36<00:00,  1.62s/it]


Since we made sample of 100 nodes from the input category now we have to remove other articles in the category **['Windows_games'].**

In order to do that we performed the following steps:

1. Making of a dictionary **d** to store value of shortest paths, where key=node, value=list of shortest paths distances
2. Making of a dictionary **z** to store value of **MINIMUM shortest path**, where key=node, value=minimum of the shortest path
3. Making of a **categories_edited**, whose category **'Windows_games'** will be consisted only of the **sampled articles**.
4. Making of a **cat_inv_dic** where key=name of the category, value=shortest path values

In [164]:
d = defaultdict(list)
for node in l:
    for key, value in node.items():
        d[key].append(value)

In [165]:
z = defaultdict(list)
for key, value in d.items():
    z[key] = np.min(d[key])

In [166]:
categories_edited = categories.copy()

In [167]:
to_delete_article = []
for article in categories_edited['American_film_actors']:
    if article not in cat_nodes_lst:
        to_delete_article.append(article)

In [168]:
for article in to_delete_article:
    categories_edited['American_film_actors'].remove(article)

As we can see now 'Windows_games' has **100 articles** the one that we sampled from the beginning.

In [169]:
len(categories_edited['American_film_actors'])

100

In [170]:
len(categories_edited.keys())

29

In [171]:
cat_inv_dic=defaultdict(list)
for cat_k,cat_v in tqdm(categories_edited.items()):
    for node,val in z.items():
        if node in cat_v:
            cat_inv_dic[cat_k].append(val)

100%|██████████████████████████████████████████████████████████████████████████████████| 29/29 [00:02<00:00, 14.19it/s]


In [172]:
len(cat_inv_dic['American_film_actors'])

100

We didn't consider the calculation of the shortest paths when 2 nodes are not connected, where the 'shortest path' would have been infinite. Since infinite doesn't make sense for median calculation we thought we could add a big number, such as 100.
So where the edges between 2 nodes don't exist we extend it to more 100 values so it is considered in the median calculation.

In [173]:
for cat_k in categories_edited.keys():
    cat_inv_dic[cat_k].extend([100]*(len(categories_edited[cat_k])-len(cat_inv_dic[cat_k])))

In [174]:
for key, value in cat_inv_dic.items():
    cat_inv_dic[key] = statistics.median(cat_inv_dic[key])

In [175]:
#cat_inv_dic, where key=name of the category, value is the med
cat_inv_dic

defaultdict(list,
            {'English_footballers': 6.0,
             'The_Football_League_players': 6.0,
             'Association_football_forwards': 8,
             'Association_football_goalkeepers': 100,
             'Association_football_midfielders': 100,
             'Association_football_defenders': 11.0,
             'Living_people': 5.0,
             'Harvard_University_alumni': 5,
             'Major_League_Baseball_pitchers': 5.0,
             'Members_of_the_United_Kingdom_Parliament_for_English_constituencies': 5,
             'Indian_films': 4.0,
             'Year_of_death_missing': 100.0,
             'Year_of_birth_missing_(living_people)': 6.0,
             'Rivers_of_Romania': 6,
             'Main_Belt_asteroids': 100.0,
             'Asteroids_named_for_people': 100,
             'English-language_albums': 4.0,
             'British_films': 3.0,
             'English-language_films': 3,
             'American_films': 3,
             'People_from_New_York_City':

In [176]:
cat_inv_dic_rank= sorted(cat_inv_dic.items(), key=operator.itemgetter(1), reverse= False)

In [177]:
#Block ranking vector
cat_inv_dic_rank

[('American_film_actors', 0.0),
 ('British_films', 3.0),
 ('English-language_films', 3),
 ('American_films', 3),
 ('American_television_actors', 3),
 ('Black-and-white_films', 3),
 ('Indian_films', 4.0),
 ('English-language_albums', 4.0),
 ('People_from_New_York_City', 4.0),
 ('Debut_albums', 4),
 ('Living_people', 5.0),
 ('Harvard_University_alumni', 5),
 ('Major_League_Baseball_pitchers', 5.0),
 ('Members_of_the_United_Kingdom_Parliament_for_English_constituencies', 5),
 ('Place_of_birth_missing_(living_people)', 5.0),
 ('American_military_personnel_of_World_War_II', 5.0),
 ('Windows_games', 5.0),
 ('English_footballers', 6.0),
 ('The_Football_League_players', 6.0),
 ('Year_of_birth_missing_(living_people)', 6.0),
 ('Rivers_of_Romania', 6),
 ('Association_football_forwards', 8),
 ('Association_football_defenders', 11.0),
 ('Association_football_goalkeepers', 100),
 ('Association_football_midfielders', 100),
 ('Year_of_death_missing', 100.0),
 ('Main_Belt_asteroids', 100.0),
 ('Astero

In [178]:
columns = ['Category name', 'Median']

block_ranking_df = pd.DataFrame([x for x in cat_inv_dic_rank], columns=columns)

In [179]:
block_ranking_df.index.names = ['Ranking']

In [180]:
block_ranking_df

,Category name,Median
Ranking,,
0,American_film_actors,0.0
1,British_films,3.0
2,English-language_films,3.0
3,American_films,3.0
4,American_television_actors,3.0
5,Black-and-white_films,3.0
6,Indian_films,4.0
7,English-language_albums,4.0
8,People_from_New_York_City,4.0


As we can see 'Windows_games' is our input category and it has median 0 which is logical cause it is an input category and shortest distances are 0 between that category and its nodes.

In [258]:
list(block_ranking_df['Category name'])

['American_film_actors',
 'British_films',
 'English-language_films',
 'American_films',
 'American_television_actors',
 'Black-and-white_films',
 'Indian_films',
 'English-language_albums',
 'People_from_New_York_City',
 'Debut_albums',
 'Living_people',
 'Harvard_University_alumni',
 'Major_League_Baseball_pitchers',
 'Members_of_the_United_Kingdom_Parliament_for_English_constituencies',
 'Place_of_birth_missing_(living_people)',
 'American_military_personnel_of_World_War_II',
 'Windows_games',
 'English_footballers',
 'The_Football_League_players',
 'Year_of_birth_missing_(living_people)',
 'Rivers_of_Romania',
 'Association_football_forwards',
 'Association_football_defenders',
 'Association_football_goalkeepers',
 'Association_football_midfielders',
 'Year_of_death_missing',
 'Main_Belt_asteroids',
 'Asteroids_named_for_people',
 'Year_of_birth_missing']

In [259]:
#Read the names of the articles from the file

In [260]:
article_names_dict={}
with open('wiki-topcats-page-names.txt') as file:
    for line in file:
        id_name=line.rstrip().split(' ')
        article_names_dict[int(id_name[0])]=" ".join(id_name[1:])

In [261]:
article_names_dict[0]

'Chiasmal syndrome'

In [262]:
#list of ranked categories
cat_lst=list(block_ranking_df['Category name'])

[STEP1] Compute subgraph induced by input category-C0. For each node compute the sum of the weigths of the in-edges.

In [263]:
input_category=cat_lst[0]

sub_graph_Cat=Gtup_directed.subgraph((list(categories_edited[input_category])))#make a subgraph containing just THAT category
in_edge_All=[] #list of first 3 sorted nodes for each category 
dict_previous_cat={} #dictionary containing a previously ranked category of the current category,key=node, value=number of in-edges 

#just for first category-input category C0
in_edge_temp_dict = defaultdict(int) #temporary dictionary to store as key=node, value=number of in-edges 
for tupla in list(sub_graph_Cat.edges()): # tupla-edge tuple, (source node, destination node)
    if tupla[1] in categories_dict[input_category]: # if the destination node belongs to the current C0 category
        in_edge_temp_dict[tupla[1]] += 1 #add one to that destination node in order to count all the in-edges

sorted_in_edge_dict= sorted(in_edge_temp_dict.items(), key=operator.itemgetter(1), reverse= True) # sort the nodes in that category
in_edge_All.append(sorted_in_edge_dict[0:3]) #add sorted dictionary of nodes to a list 
#first2pairs = {k: mydict[k] for k in list(mydict)[:2]

In [264]:
#one edge therefore one article
in_edge_All

[[(1181827, 6), (1061932, 2), (1061429, 2)]]

[STEP2] Extend the graph to the nodes that belong to C1. Thus, for each article in C1 compute the score as before. Note that the in-edges coming from the previous category, C0, have as weights the score of the node that sends the edge.

[STEP3] Repeat Step2 up to the last category of the ranking. In the last step of the example you clearly see the weight update of the edge coming from node E.

In [265]:
print(len(list(sub_graph_Cat.edges())))

13


In [266]:
a={}
a[0]=1
a[1]=1
b={}
b[0]=2
b[2]=1

In [267]:
a.update(b)
a

{0: 2, 1: 1, 2: 1}

In [268]:
len(cat_lst)

29

In [269]:
in_edge_dict=in_edge_temp_dict #take nodes and values from the 1st input category
#save previous category to a dict_previous_cat dictionary as a temporary variable
for cat in tqdm(cat_lst[1:]):#for other 28 categories
    in_edge_temp_dict = defaultdict(int)
    sub_graph_Cat=Gtup_directed.subgraph((list(categories_edited[cat])))#make a subgraph containing just THAT category
    for tupla in list(sub_graph_Cat.edges()): # go to every node of the cat subgraph
        try:
            source=in_edge_dict[tupla[0]] #take the source node from the previous category
            in_edge_temp_dict[tupla[1]] += source #add in-edge value from the previous node
        except:
            in_edge_temp_dict[tupla[1]] += 1 #append 1 if it is not there
    in_edge_dict.update(in_edge_temp_dict)

    sorted_in_edge_dict= sorted(in_edge_temp_dict.items(), key=operator.itemgetter(1), reverse= True)
    in_edge_All.append(sorted_in_edge_dict[0:3])

100%|██████████████████████████████████████████████████████████████████████████████████| 28/28 [00:12<00:00,  6.68it/s]


In [270]:
len(in_edge_All)

29

In [271]:
in_edge_All

[[(1181827, 6), (1061932, 2), (1061429, 2)],
 [(1056770, 0), (1056769, 0), (1041937, 0)],
 [(1062743, 0), (1064184, 0), (1064265, 0)],
 [(1245946, 0), (688149, 0), (688300, 0)],
 [(1061913, 3), (1061855, 2), (1061770, 2)],
 [(589847, 0), (590961, 0), (1063431, 0)],
 [(582698, 0), (590800, 0), (589828, 0)],
 [(1228798, 0), (1228799, 0), (145526, 0)],
 [(1224802, 2), (1061452, 2), (1224704, 2)],
 [(566701, 0), (328881, 0), (155018, 0)],
 [(1061902, 12), (1061932, 12), (1061913, 11)],
 [(1400478, 2), (1400534, 2), (245119, 2)],
 [(386079, 0), (384805, 0), (387201, 0)],
 [(536243, 0), (543166, 0), (543175, 0)],
 [(1365478, 2), (1365479, 2), (1365477, 2)],
 [(1400484, 4), (1400483, 4), (1061812, 4)],
 [(1734405, 0)],
 [(80379, 0), (87391, 0), (78944, 0)],
 [(81787, 0), (81941, 0), (82484, 0)],
 [(643206, 0), (973046, 0), (655275, 0)],
 [(785340, 0), (786385, 0), (786433, 0)],
 [(1358244, 0), (88262, 0), (884966, 0)],
 [(79221, 0), (81524, 0), (81919, 0)],
 [(737293, 0), (81952, 0), (81104, 

In [272]:
l=[*in_edge_All]

In [273]:
for idx,row in enumerate(in_edge_All):
    print(row)

[(1181827, 6), (1061932, 2), (1061429, 2)]
[(1056770, 0), (1056769, 0), (1041937, 0)]
[(1062743, 0), (1064184, 0), (1064265, 0)]
[(1245946, 0), (688149, 0), (688300, 0)]
[(1061913, 3), (1061855, 2), (1061770, 2)]
[(589847, 0), (590961, 0), (1063431, 0)]
[(582698, 0), (590800, 0), (589828, 0)]
[(1228798, 0), (1228799, 0), (145526, 0)]
[(1224802, 2), (1061452, 2), (1224704, 2)]
[(566701, 0), (328881, 0), (155018, 0)]
[(1061902, 12), (1061932, 12), (1061913, 11)]
[(1400478, 2), (1400534, 2), (245119, 2)]
[(386079, 0), (384805, 0), (387201, 0)]
[(536243, 0), (543166, 0), (543175, 0)]
[(1365478, 2), (1365479, 2), (1365477, 2)]
[(1400484, 4), (1400483, 4), (1061812, 4)]
[(1734405, 0)]
[(80379, 0), (87391, 0), (78944, 0)]
[(81787, 0), (81941, 0), (82484, 0)]
[(643206, 0), (973046, 0), (655275, 0)]
[(785340, 0), (786385, 0), (786433, 0)]
[(1358244, 0), (88262, 0), (884966, 0)]
[(79221, 0), (81524, 0), (81919, 0)]
[(737293, 0), (81952, 0), (81104, 0)]
[(78944, 0), (88754, 0), (82393, 0)]
[(1048

In [274]:
for idx,row in enumerate(in_edge_All):
    row_lst=[]
    for tup in row:
        tuple_new=(article_names_dict[tup[0]],tup[1])
        row_lst.append(tuple_new)
    print(idx,cat_lst[idx],row_lst)

0 American_film_actors [('Elvis Presley', 6), ('Jennifer Jason Leigh', 2), ('Lee Marvin', 2)]
1 British_films [('The Golden Voyage of Sinbad', 0), ('Sinbad and the Eye of the Tiger', 0), ('Cinema of the United Kingdom', 0)]
2 English-language_films [('Reservoir Dogs', 0), ('Predators (film)', 0), ('Machete (film)', 0)]
3 American_films [('Educating Peter', 0), ('Behind the Green Door', 0), ('Deep Throat (film)', 0)]
4 American_television_actors [('Sean Penn', 3), ('Forest Whitaker', 2), ('Ron Howard', 2)]
5 Black-and-white_films [('Ladki', 0), ('Penn (film)', 0), ('Duck Soup (1933 film)', 0)]
6 Indian_films [('Shola Aur Shabnam (1992 film)', 0), ('Illarikam', 0), ('Aasha (1957 film)', 0)]
7 English-language_albums [('Bloodrock U.S.A.', 0), ('Bloodrock 2', 0), ('Wild in the Streets (Circle Jerks album)', 0)]
8 People_from_New_York_City [("Eugene O'Neill", 2), ('Lauren Bacall', 2), ('Arthur Miller', 2)]
9 Debut_albums [('America (America album)', 0), ('No One Can Do It Better', 0), ('The